# Módulo de separación de imágenes.

# Importar librerías y repositorio del proyecto

In [6]:
import torch
from torch import nn

In [7]:
!git clone https://github.com/stefanoschiappacasse/ProyectoViT

Cloning into 'ProyectoViT'...
remote: Enumerating objects: 193, done.
remote: Counting objects: 100% (193/193), done.
remote: Compressing objects: 100% (164/164), done.
remote: Total 193 (delta 45), reused 167 (delta 24), pack-reused 0
Receiving objects: 100% (193/193), 1.05 MiB | 15.21 MiB/s, done.
Resolving deltas: 100% (45/45), done.


In [8]:
cd /content/ProyectoViT

/content/ProyectoViT


In [9]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Data Set Yoga 82

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
!unzip '/content/drive/MyDrive/MDS/2022-2/Procesamiento avanzado de imágenes/Proyecto/Yoga-82.zip'

Streaming output truncated to the last 5000 lines.
 extracting: Yoga-82/Images/Supta_Virasana_Vajrasana/4_47.jpg  
  inflating: Yoga-82/Images/Supta_Virasana_Vajrasana/4_48.jpg  
  inflating: Yoga-82/Images/Supta_Virasana_Vajrasana/4_5.jpg  
  inflating: Yoga-82/Images/Supta_Virasana_Vajrasana/4_50.jpg  
 extracting: Yoga-82/Images/Supta_Virasana_Vajrasana/4_51.jpg  
  inflating: Yoga-82/Images/Supta_Virasana_Vajrasana/4_52.jpg  
  inflating: Yoga-82/Images/Supta_Virasana_Vajrasana/4_53.jpg  
  inflating: Yoga-82/Images/Supta_Virasana_Vajrasana/4_54.jpg  
  inflating: Yoga-82/Images/Supta_Virasana_Vajrasana/4_55.jpg  
 extracting: Yoga-82/Images/Supta_Virasana_Vajrasana/4_56.jpg  
  inflating: Yoga-82/Images/Supta_Virasana_Vajrasana/4_57.jpg  
  inflating: Yoga-82/Images/Supta_Virasana_Vajrasana/4_58.jpg  
  inflating: Yoga-82/Images/Supta_Virasana_Vajrasana/4_61.jpg  
  inflating: Yoga-82/Images/Supta_Virasana_Vajrasana/4_65.jpg  
 extracting: Yoga-82/Images/Supta_Virasana_Vajrasana/4

# Entrenamiento

Se utiliza otra función de entrenamiento, distinta a la del repositorio debido a que se iban guardando los resultados en la sesión de *Weight and biases*.

In [13]:
def train_model(model, 
                criterio, 
                optimizer,
                scheduler, 
                project_name,
                experiment_name,
                lr,
                weight_decay,
                arquitecture_name,
                dataset_name,
                batch_size,
                optim,
                chkp_path,
                num_epochs = 25):
  """
  Función que aplica el entrenamiento del modelo, 
  dado un criterio de loss, un optimizador y la cantidad de épocas.
  Retorna el mejor modelo encontrado, en conjunto con las losses de 
  entrenamiento y validación, como también los accuracy para ambos conjuntos.
  """
  since = time.time()
  
  best_model_wts = copy.deepcopy(model.state_dict())
  best_acc = 0.0
  best_acc_train = 0.0
  val_acc = []
  train_acc = []
  val_loss = []
  train_loss = []

  wandb.init(
      # Set the project where this run will be logged
      project=project_name, 
      # We pass a run name (otherwise it’ll be randomly assigned, like sunshine-lollypop-10)
      name=experiment_name, 
      # Track hyperparameters and run metadata
      config={
      "learning_rate": lr,
      "weight_decay":weight_decay,
      "architecture": arquitecture_name,
      "dataset": dataset_name,
      "epochs": num_epochs,
      "batch_size": batch_size,
      "optimizer": optim
      })
  
  early_stopper = EarlyStopper(patience=5)

  for epoch in range(num_epochs):
    print('Epoch {}/{}'.format(epoch, num_epochs-1))
    print('-' * 10)
    
    #Train model
    scheduler.step()
    model.train()
    
    running_loss = 0.0
    running_corrects = 0.0
    
    for inputs, labels in train_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)
          
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)
    
    epoch_loss = running_loss / len(train_loader.dataset)
    epoch_acc = running_corrects.double() / len(train_loader.dataset)
      
    print('Train Loss: {:.4f} Acc: {:.4f}'.format(epoch_loss, epoch_acc))
    train_loss.append(epoch_loss)
    train_acc.append(epoch_acc)
    epoch_acc_train = epoch_acc
    
    wandb.log({"acc_train": epoch_acc, "loss_train": epoch_loss})

    #Validation 
    model.eval()
    running_loss = 0.0
    running_corrects = 0.0
    
    for inputs, labels in val_loader:
      inputs = inputs.to(device)
      labels = labels.to(device)
        
      with torch.set_grad_enabled(False):
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)
          
      running_loss += loss.item() * inputs.size(0)
      running_corrects += torch.sum(preds == labels.data)
     
    epoch_loss = running_loss / len(val_loader.dataset)
    epoch_acc = running_corrects.double() / len(val_loader.dataset)

    wandb.log({"acc_val": epoch_acc, "loss_val": epoch_loss})
      
    print('Val Loss: {:.4f} Acc: {:.4f}'.format(epoch_loss, epoch_acc))
    val_loss.append(epoch_loss)
    val_acc.append(epoch_acc)
    
    if epoch_acc > best_acc:
        best_acc = epoch_acc
        best_acc_train = epoch_acc_train
        best_model_wts = copy.deepcopy(model.state_dict())

    if early_stopper.early_stop(epoch_loss):             
        break
    if (epoch + 1)%5 == 0:
      torch.save(mynet.state_dict(), chkp_path + '_' + str((epoch + 1)) +'.pth')
        
  time_elapsed = time.time() - since
  print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed//60, 
                                                      time_elapsed % 60))
  print('Best train accucary: {:.4f}'.format(best_acc_train))
  print('Best val accucary: {:.4f}'.format(best_acc))

  model.load_state_dict(best_model_wts)
  return model, train_loss, train_acc, val_loss, val_acc

## Entrenamiento 6 clases

### Entrenamiento inicial 

In [12]:
from utils.dataset import Yoga_82 
from utils.data_loading import create_dataloaders 
from ViT.ViT_model import ViT

In [ ]:
#creación dataset
train_dataset = Yoga_82(root = 'Yoga-82/', 
                        file_ = 'yoga_train.txt', 
                        transform = True, 
                        prueba = False, 
                        nivel = 6)
#creación dataloaders
train_loader, val_loader = create_dataloaders(dataset = train_dataset, 
                                              set_ = 'train', 
                                              batch_size = 64, 
                                              val_size =0.2)

In [ ]:
#Entrenamiento red inicial
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
mynet = ViT(n_classes=6, emb_size = 768, num_heads = 12)
mynet.to(device=device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(mynet.parameters(), lr=0.0001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

mynet, mynet_tl, mynet_ta, mynet_vl, mynet_va = \
  train_model(mynet, 
              criterion, 
              optimizer, 
              scheduler = scheduler,
              project_name = 'ViT-EL7008',
              experiment_name = 'Class level 6 - sin transfer learning',
              lr = '0.0001',
              weight_decay = '0.0',
              arquitecture_name = 'ViT',
              dataset_name = 'Yoga-82',
              batch_size = 64,
              optim = 'Adam',
              num_epochs = 20)

# Entrenamiento con 20 clases - sin pretrained

In [ ]:
#creación dataset
train_dataset = Yoga_82(root = 'Yoga-82/', 
                        file_ = 'yoga_train.txt', 
                        transform = True, 
                        prueba = False, 
                        nivel = 20)
#creación dataloaders
train_loader, val_loader = create_dataloaders(dataset = train_dataset, 
                                              set_ = 'train', 
                                              batch_size = 64, 
                                              val_size =0.2)

In [ ]:
#Entrenamiento red 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
mynet = ViT(n_classes=20, emb_size = 768, num_heads = 12)
mynet.to(device=device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(mynet.parameters(), lr=0.0001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

mynet, mynet_tl, mynet_ta, mynet_vl, mynet_va = \
  train_model(mynet, 
              criterion, 
              optimizer, 
              scheduler,
              project_name = 'ViT-EL7008',
              experiment_name = 'Class level 20 - sin transfer learning',
              lr = '0.0001',
              weight_decay = '0.0',
              arquitecture_name = 'ViT',
              dataset_name = 'Yoga-82',
              batch_size = 64,
              optim = 'Adam',
              chkp_path = '/content/drive/MyDrive/MDS/Vit_20_sin_transfer_learning',
              num_epochs = 20)

wandb: Currently logged in as: stefanosch. Use `wandb login --relogin` to force relogin


Epoch 0/19
----------


/usr/local/lib/python3.8/dist-packages/torch/optim/lr_scheduler.py:138: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


Train Loss: 2.9567 Acc: 0.0972
Val Loss: 2.8783 Acc: 0.0985
Epoch 1/19
----------
Train Loss: 2.8923 Acc: 0.0990
Val Loss: 2.8673 Acc: 0.1065
Epoch 2/19
----------
Train Loss: 2.8521 Acc: 0.1148
Val Loss: 2.8391 Acc: 0.1220
Epoch 3/19
----------
Train Loss: 2.8096 Acc: 0.1261
Val Loss: 2.8023 Acc: 0.1276
Epoch 4/19
----------
Train Loss: 2.7751 Acc: 0.1428
Val Loss: 2.7574 Acc: 0.1293
Epoch 5/19
----------
Train Loss: 2.7468 Acc: 0.1486
Val Loss: 2.7428 Acc: 0.1604
Epoch 6/19
----------
Train Loss: 2.6886 Acc: 0.1722
Val Loss: 2.6871 Acc: 0.1723
Epoch 7/19
----------
Train Loss: 2.6719 Acc: 0.1761
Val Loss: 2.6717 Acc: 0.1746
Epoch 8/19
----------
Train Loss: 2.6492 Acc: 0.1867
Val Loss: 2.6581 Acc: 0.1789
Epoch 9/19
----------
Train Loss: 2.6400 Acc: 0.1859
Val Loss: 2.6487 Acc: 0.1796
Epoch 10/19
----------
Train Loss: 2.6246 Acc: 0.1917
Val Loss: 2.6480 Acc: 0.1872
Epoch 11/19
----------
Train Loss: 2.6160 Acc: 0.1966
Val Loss: 2.6389 Acc: 0.1855
Epoch 12/19
----------
Train Loss: 2

## Entrenamiento 20 clases con modelo pre entrenado con 6 clases

In [ ]:
def load_checkpoint(model, filepath):
    checkpoint_state_dict = torch.load(filepath)
    model.load_state_dict(checkpoint_state_dict)
    
    return model

In [ ]:
#creación dataset
train_dataset = Yoga_82(root = 'Yoga-82/', 
                        file_ = 'yoga_train.txt', 
                        transform = True, 
                        prueba = False, 
                        nivel = 20)
#creación dataloaders
train_loader, val_loader = create_dataloaders(dataset = train_dataset, 
                                              set_ = 'train', 
                                              batch_size = 64, 
                                              val_size =0.2)

In [ ]:
ruta_chkp = '/content/drive/MyDrive/MDS/ViT_6_checkpoint.pth'

In [ ]:
mynet = ViT(n_classes=6, emb_size = 768, num_heads = 12)
mynet = load_checkpoint(mynet, ruta_chkp)
mynet.clf = ViT.ViT_parts.ClassificationHead(768, 20)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
mynet.to(device=device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(mynet.parameters(), lr=0.0001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

mynet, mynet_tl, mynet_ta, mynet_vl, mynet_va = \
  train_model(mynet, 
              criterion, 
              optimizer, 
              scheduler,
              project_name = 'ViT-EL7008',
              experiment_name = 'Class level 20 - transfer learning - oficial',
              lr = '0.0001',
              weight_decay = '0.0',
              arquitecture_name = 'ViT',
              dataset_name = 'Yoga-82',
              batch_size = 64,
              optim = 'Adam',
              chkp_path = '/content/drive/MyDrive/MDS/Vit_20_transfer_learning',
              num_epochs = 20)

wandb: Currently logged in as: stefanosch. Use `wandb login --relogin` to force relogin


Epoch 0/19
----------


/usr/local/lib/python3.8/dist-packages/torch/optim/lr_scheduler.py:138: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


Train Loss: 2.6826 Acc: 0.1709
Val Loss: 2.6493 Acc: 0.1835
Epoch 1/19
----------
Train Loss: 2.6029 Acc: 0.2008
Val Loss: 2.6144 Acc: 0.2001
Epoch 2/19
----------
Train Loss: 2.5751 Acc: 0.2112
Val Loss: 2.6003 Acc: 0.2126
Epoch 3/19
----------
Train Loss: 2.5554 Acc: 0.2128
Val Loss: 2.5926 Acc: 0.2123
Epoch 4/19
----------
Train Loss: 2.5425 Acc: 0.2185
Val Loss: 2.5847 Acc: 0.2057
Epoch 5/19
----------
Train Loss: 2.5215 Acc: 0.2217
Val Loss: 2.5503 Acc: 0.2259
Epoch 6/19
----------
Train Loss: 2.4132 Acc: 0.2571
Val Loss: 2.4863 Acc: 0.2526
Epoch 7/19
----------
Train Loss: 2.3799 Acc: 0.2703
Val Loss: 2.4637 Acc: 0.2520
Epoch 8/19
----------
Train Loss: 2.3520 Acc: 0.2787
Val Loss: 2.4521 Acc: 0.2606
Epoch 9/19
----------
Train Loss: 2.3416 Acc: 0.2792
Val Loss: 2.4644 Acc: 0.2573
Epoch 10/19
----------
Train Loss: 2.3240 Acc: 0.2896
Val Loss: 2.4567 Acc: 0.2569
Epoch 11/19
----------
Train Loss: 2.3126 Acc: 0.2882
Val Loss: 2.4329 Acc: 0.2642
Epoch 12/19
----------
Train Loss: 2

# Entrenamiento con 82 clases - sin transfer learning

In [ ]:
#creación dataset
train_dataset = Yoga_82(root = 'Yoga-82/', 
                        file_ = 'yoga_train.txt', 
                        transform = True, 
                        prueba = False, 
                        nivel = 82)
#creación dataloaders
train_loader, val_loader = create_dataloaders(dataset = train_dataset, 
                                              set_ = 'train', 
                                              batch_size = 64, 
                                              val_size =0.2)

In [ ]:
#Entrenamiento red 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
mynet = ViT(n_classes=82, emb_size = 768, num_heads = 12)
mynet.to(device=device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(mynet.parameters(), lr=0.0001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

mynet, mynet_tl, mynet_ta, mynet_vl, mynet_va = \
  train_model(mynet, 
              criterion, 
              optimizer, 
              scheduler,
              project_name = 'ViT-EL7008',
              experiment_name = 'Class level 82 - sin transfer learning',
              lr = '0.0001',
              weight_decay = '0.0',
              arquitecture_name = 'ViT',
              dataset_name = 'Yoga-82',
              batch_size = 128,
              optim = 'Adam',
              chkp_path = '/content/drive/MyDrive/MDS/Vit_82_sin_transfer_learning',
              num_epochs = 20)

wandb: Currently logged in as: stefanosch. Use `wandb login --relogin` to force relogin


Epoch 0/19
----------


/usr/local/lib/python3.8/dist-packages/torch/optim/lr_scheduler.py:138: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


Train Loss: 4.3457 Acc: 0.0371
Val Loss: 4.2793 Acc: 0.0453
Epoch 1/19
----------
Train Loss: 4.2996 Acc: 0.0383
Val Loss: 4.2841 Acc: 0.0446
Epoch 2/19
----------
Train Loss: 4.2720 Acc: 0.0389
Val Loss: 4.2350 Acc: 0.0453
Epoch 3/19
----------
Train Loss: 4.2366 Acc: 0.0386
Val Loss: 4.2468 Acc: 0.0407
Epoch 4/19
----------
Train Loss: 4.2078 Acc: 0.0400
Val Loss: 4.2079 Acc: 0.0324
Epoch 5/19
----------
Train Loss: 4.1742 Acc: 0.0460
Val Loss: 4.1586 Acc: 0.0489
Epoch 6/19
----------
Train Loss: 4.1086 Acc: 0.0567
Val Loss: 4.1016 Acc: 0.0565
Epoch 7/19
----------
Train Loss: 4.0816 Acc: 0.0566
Val Loss: 4.0923 Acc: 0.0559
Epoch 8/19
----------
Train Loss: 4.0652 Acc: 0.0588
Val Loss: 4.0772 Acc: 0.0608
Epoch 9/19
----------
Train Loss: 4.0536 Acc: 0.0613
Val Loss: 4.0776 Acc: 0.0645
Epoch 10/19
----------
Train Loss: 4.0442 Acc: 0.0607
Val Loss: 4.0641 Acc: 0.0592
Epoch 11/19
----------
Train Loss: 4.0343 Acc: 0.0614
Val Loss: 4.0644 Acc: 0.0618
Epoch 12/19
----------
Train Loss: 4

# Entrenamiento con 82 clases - con transfer learning

In [ ]:
#creación dataset
train_dataset = Yoga_82(root = 'Yoga-82/', 
                        file_ = 'yoga_train.txt', 
                        transform = True, 
                        prueba = False, 
                        nivel = 82)
#creación dataloaders
train_loader, val_loader = create_dataloaders(dataset = train_dataset, 
                                              set_ = 'train', 
                                              batch_size = 64, 
                                              val_size =0.2)

In [ ]:
ruta_chkp = '/content/drive/MyDrive/MDS/ViT_20_checkpoint.pth'

In [ ]:
mynet = ViT(n_classes=20, emb_size = 768, num_heads = 12)
mynet = load_checkpoint(mynet, ruta_chkp)
mynet.clf = ViT.ViT_parts.ClassificationHead(768, 82)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
mynet.to(device=device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(mynet.parameters(), lr=0.0001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

mynet, mynet_tl, mynet_ta, mynet_vl, mynet_va = \
  train_model(mynet, 
              criterion, 
              optimizer, 
              scheduler,
              project_name = 'ViT-EL7008',
              experiment_name = 'Class level 82 - transfer learning',
              lr = '0.0001',
              weight_decay = '0.0',
              arquitecture_name = 'ViT',
              dataset_name = 'Yoga-82',
              batch_size = 64,
              optim = 'Adam',
              chkp_path = '/content/drive/MyDrive/MDS/Vit_82_transfer_learning',
              num_epochs = 20)

Epoch 0/19
----------
Train Loss: 3.9386 Acc: 0.0893
Val Loss: 3.8203 Acc: 0.1019
Epoch 1/19
----------
Train Loss: 3.7486 Acc: 0.1242
Val Loss: 3.7938 Acc: 0.1151
Epoch 2/19
----------
Train Loss: 3.7034 Acc: 0.1244
Val Loss: 3.7422 Acc: 0.1286
Epoch 3/19
----------
Train Loss: 3.6339 Acc: 0.1370
Val Loss: 3.6658 Acc: 0.1362
Epoch 4/19
----------
Train Loss: 3.6036 Acc: 0.1440
Val Loss: 3.6925 Acc: 0.1303
Epoch 5/19
----------
Train Loss: 3.5591 Acc: 0.1548
Val Loss: 3.6666 Acc: 0.1313
Epoch 6/19
----------
Train Loss: 3.3421 Acc: 0.1947
Val Loss: 3.5419 Acc: 0.1594
Epoch 7/19
----------
Train Loss: 3.2708 Acc: 0.2046
Val Loss: 3.5122 Acc: 0.1723
Epoch 8/19
----------
Train Loss: 3.2324 Acc: 0.2116
Val Loss: 3.5017 Acc: 0.1627
Epoch 9/19
----------
Train Loss: 3.2075 Acc: 0.2208
Val Loss: 3.5146 Acc: 0.1713
Epoch 10/19
----------
Train Loss: 3.1720 Acc: 0.2236
Val Loss: 3.4863 Acc: 0.1736
Epoch 11/19
----------
Train Loss: 3.1555 Acc: 0.2294
Val Loss: 3.4828 Acc: 0.1786
Epoch 12/19
--